In [1]:
from csg.core import CSG
from csg.geom import Vertex, Vector, Plane, Polygon
import meshplot as mp
import numpy as np
import geo_vase

def plotCsg(csg):
    verts, polys, count = csg.toVerticesAndPolygons()
    triangles = []

    tris = []
    # polys are always convex (apparently...)
    for poly in polys:
        vlen = len(poly)
        for n in range(1,vlen-1):
            tris.append(
                [
                    poly[0],
                    poly[n%vlen],
                    poly[(n+1)%vlen],
                ]
            )
    mp.plot(np.array(verts), np.array(tris), shading={"point_size": 0.1})

gv = geo_vase.GeoVase(
    rows=7,
    sides=5
)

# Nice --- easy to convert from our mesh to CSG mesh.
ps = CSG.fromPolygons([Polygon([Vertex(gv.verts[v].tolist()) for v in reversed(f)]) for f in gv.faces])
b = CSG.fromPolygons([Polygon([Vertex(gv.verts[v].tolist()) for v in reversed(f)]) for f in gv.faces])
b.rotate([0, 1, 0], 30)

plotCsg(ps.intersect(b))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0017208…

In [3]:
def frameCube(center, frame, scale):
    n, u, v = frame
    verts = []
    
    for i in range(8):
        a = (2 * bool(i & 1) - 1)
        b = (2 * bool(i & 2) - 1)
        c = (2 * bool(i & 4) - 1)

        verts.append(Vertex((center - (n*a + u*b + v*c)*scale).tolist()))
    polys = []
    cubeFaces = [
        [[0, 4, 6, 2], [-1, 0, 0]],
        [[1, 3, 7, 5], [+1, 0, 0]],
        [[0, 1, 5, 4], [0, -1, 0]],
        [[2, 6, 7, 3], [0, +1, 0]],
        [[0, 2, 3, 1], [0, 0, -1]],
        [[4, 5, 7, 6], [0, 0, +1]]
    ]
    
    for f,_ in cubeFaces:
        polys.append(Polygon([ verts[v] for v in f]))

    return CSG.fromPolygons(polys)



gv = geo_vase.GeoVase(
    rows=11,
    sides=7
)

c = CSG.cube(center=[0,0.65,0], radius=[10, 0.55, 10])

for i, vert in enumerate(gv.verts):
    vnorms = gv.vnorms[i]
    n, u, v = vnorms
    r = 0.05
    q = 10
    c = c - frameCube(vert - n*q, vnorms, q + r)


plotCsg(c)
verts, polys, count = c.toVerticesAndPolygons()
print({
    "vertices": verts,
    "faces": [[x for x in reversed(p)] for p in polys]
})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0039288…

{'verts': [(-0.3882212544999999, 0.22575205180000002, 0.10882729580000006), (-0.49441745857999986, 0.3996130939000002, 0.09102012770000001), (-0.49441745857999986, 0.3996130939000002, -0.09102012769999979), (-0.3882212544999999, 0.22575205180000002, -0.10882729579999983), (-0.31141054175999994, 0.10000000000000009, -0.06544614629999979), (-0.31141054175999994, 0.10000000000000009, 0.06544614630000001), (0.34071611720000017, 1.2, 0.12434754600000009), (0.4515207295000001, 1.1573828414000003, 0.049904329900000155), (0.4515207295000001, 1.1573828414000003, -0.04990432989999993), (0.34071611720000017, 1.2, -0.12434754599999986), (0.11521419820000012, 1.2, 0.34391201390000004), (0.24250179400000005, 1.1573828414000003, 0.3841279621000002), (0.32053534640000003, 1.1573828414000003, 0.32189828060000014), (0.3096518507000001, 1.2, 0.1888531602000001), (-0.1970463619999998, 1.2, 0.3045037207000001), (-0.1491259385999999, 1.1573828414000003, 0.42909540410000013), (-0.05181969029999989, 1.1573828